In [5]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [58]:
import json 
from pyspark.sql import functions as F
from pyspark.sql import SparkSession 

In [6]:
user_id = 592

In [31]:
rdd_data = sc.textFile('ml-100k/u.data')
rdd_data = rdd_data.map(lambda x: x.split('\t')).map(lambda x: (int(x[0]),int(x[1]),int(x[2])))
rdd.cache()

PythonRDD[38] at RDD at PythonRDD.scala:48

# Часть1. Основные характеристики данных MovieLens

Найдите количество всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).

In [49]:
df_data = rdd_data.toDF(['user_id','item_id','rating'])

In [79]:
df_data.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           user_id|          item_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|            100000|           100000|            100000|
|   mean|         462.48475|        425.53013|           3.52986|
| stddev|266.61442012750945|330.7983563255858|1.1256735991443179|
|    min|                 1|                1|                 1|
|    max|               943|             1682|                 5|
+-------+------------------+-----------------+------------------+



In [69]:
user_id_desc = df_data.agg(F.countDistinct(df_data.user_id).alias('user_id_desc')).collect()
user_id_desc[0][0]

943

In [70]:
item_id_desc = df_data.agg(F.countDistinct(df_data.item_id).alias('item_id_desc')).collect()
item_id_desc[0][0]

1682

Сколько пользователь в среднем ставит рейтингов? Подсчитать количество рейтингов / количество пользователей. Поле average_user_ratings.

In [87]:
average_user_ratings = df_data.count()/float(user_id_desc[0][0])
average_user_ratings

106.04453870625663

Сколько фильм в среднем имеет рейтингов? Подсчитатьколичество рейтингов / количество фильмов. Поле average_film_ratings

In [88]:
average_film_ratings = df_data.count()/float(item_id_desc[0][0])
average_film_ratings

59.45303210463734

Найдите процент заполненных ячеек в данных: количество рейтингов / (количество пользователей * количество фильмов). Поле completeness

In [360]:
completeness = df_data.count() / float(user_id_desc[0][0]*item_id_desc[0][0])
completeness

0.06304669364224531

# Часть 2. User-user CF

Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/количество рейтингов пользователя).

In [119]:
sum_rat_user = df_data.groupBy('user_id').agg({'rating': 'sum'}).alias('sum_r')

In [125]:
count_rat_user = df_data.groupBy('user_id').agg({'rating': 'count'}).alias('count_r')

In [229]:
avg_user_rating = df_data.groupBy('user_id').avg('rating').withColumnRenamed("avg(rating)","avg_user_rating")
avg_user_rating = avg_user_rating.withColumnRenamed('user_id','user')
avg_user_rating.show(10)

+----+------------------+
|user|   avg_user_rating|
+----+------------------+
|  26|  2.94392523364486|
| 474|  4.08256880733945|
|  29|3.6470588235294117|
| 541|3.6240601503759398|
| 558|               4.2|
|  65|            3.9375|
| 418|               2.9|
| 191|3.6296296296296298|
| 293|3.0309278350515463|
| 222| 3.049095607235142|
+----+------------------+
only showing top 10 rows



In [194]:
my_user_avg_rat = avg_user_rating[avg_user_rating['user']==user_id]
my_user_avg_rat.show()

+----+-----------------+
|user|  avg_user_rating|
+----+-----------------+
| 592|3.813888888888889|
+----+-----------------+



In [232]:
df_my_user = df_data[df_data['user_id']==user_id].selectExpr('user_id as my_user_id', \
                                                             'item_id as my_item_id',\
                                                             'rating as my_user_rating')
df_my_user.show(10)

+----------+----------+--------------+
|my_user_id|my_item_id|my_user_rating|
+----------+----------+--------------+
|       592|       418|             4|
|       592|       124|             5|
|       592|        50|             5|
|       592|       204|             5|
|       592|       297|             5|
|       592|        79|             4|
|       592|       969|             4|
|       592|        20|             4|
|       592|       433|             5|
|       592|       323|             1|
+----------+----------+--------------+
only showing top 10 rows



In [364]:
df_ua = df_data.join(df_my_user, df_data['item_id']==df_my_user['my_item_id'], 'inner')

In [365]:
df_ua = df_ua.join(avg_user_rating, df_ua['user_id']==avg_user_rating['user']).\
                            select('user_id','item_id','rating','my_user_rating','avg_user_rating')
df_ua.show(10)

+-------+-------+------+--------------+----------------+
|user_id|item_id|rating|my_user_rating| avg_user_rating|
+-------+-------+------+--------------+----------------+
|     26|    628|     3|             3|2.94392523364486|
|     26|     14|     3|             5|2.94392523364486|
|     26|   1014|     3|             4|2.94392523364486|
|     26|    685|     3|             2|2.94392523364486|
|     26|    237|     3|             4|2.94392523364486|
|     26|    150|     3|             5|2.94392523364486|
|     26|    246|     4|             5|2.94392523364486|
|     26|      1|     3|             4|2.94392523364486|
|     26|    815|     2|             3|2.94392523364486|
|     26|    283|     3|             4|2.94392523364486|
+-------+-------+------+--------------+----------------+
only showing top 10 rows



In [366]:
df_ua = df_ua.select('*', (F.col('rating')-F.col('avg_user_rating')).alias('diff_user_r'),\
                  (F.col('my_user_rating')-3.813888888888889).alias('diff_my_user_r'))
df_ua.show(10)

+-------+-------+------+--------------+----------------+-------------------+-------------------+
|user_id|item_id|rating|my_user_rating| avg_user_rating|        diff_user_r|     diff_my_user_r|
+-------+-------+------+--------------+----------------+-------------------+-------------------+
|     26|   1016|     3|             4|2.94392523364486|0.05607476635513997|0.18611111110999978|
|     26|    546|     2|             4|2.94392523364486|  -0.94392523364486|0.18611111110999978|
|     26|    246|     4|             5|2.94392523364486|   1.05607476635514| 1.1861111111099998|
|     26|    248|     3|             4|2.94392523364486|0.05607476635513997|0.18611111110999978|
|     26|    257|     3|             4|2.94392523364486|0.05607476635513997|0.18611111110999978|
|     26|    405|     2|             4|2.94392523364486|  -0.94392523364486|0.18611111110999978|
|     26|    250|     3|             4|2.94392523364486|0.05607476635513997|0.18611111110999978|
|     26|    748|     1|      

In [367]:
df_ua = df_ua.select('*',(F.col('diff_user_r')*F.col('diff_my_user_r')).alias('mult_a_u'))

In [245]:
PearsonSim = df_ua.select('item_id','user_id','mult_a_u', 'diff_my_user_r', 'diff_user_r').groupby('user_id') \
    .agg(F.sum('mult_a_u').alias('sum1'), F.sqrt(F.sum(F.col('diff_my_user_r')**2)).alias('sqrt1'), \
                                          F.sqrt(F.sum(F.col('diff_user_r')**2)).alias('sqrt2'),\
                                         (F.count('item_id')/50).alias('num_50'))\
    .withColumn('PearsonSim', F.col('sum1') / (F.col('sqrt1')*F.col('sqrt2')))
PearsonSim.show(10)

+-------+-------------------+------------------+------------------+------+--------------------+
|user_id|               sum1|             sqrt1|             sqrt2|num_50|          PearsonSim|
+-------+-------------------+------------------+------------------+------+--------------------+
|     26|  37.29207791073451|13.370441781211017| 7.335900926042817|  1.62|  0.3802046010514602|
|     29| 21.179930795847746| 7.104509021945612| 4.459351202673875|  0.52|  0.6685267498903249|
|    474|  61.70128777038967| 11.80982345220386|10.905532796179877|  3.08| 0.47907547058703775|
|     65|         1.75390625|  7.18967358438476| 6.694505676299035|  0.98|  0.0364400268215457|
|    191|  8.065843621399178| 5.649574804642474|3.3487298740787264|  0.42| 0.42633787660685585|
|    418| 10.350000000000003| 6.553624951124378| 5.809475019311125|   0.3|   0.271845330262568|
|    541|  26.72078692973036| 8.907994492426385|7.8159585453328715|   0.9|  0.3837841695662868|
|    558|-2.5200000000000014|  4.2755116

In [246]:
PearsonSim = PearsonSim.withColumn('Pearson_k',F.when(PearsonSim.num_50 > 1, 1)\
                                   .otherwise(PearsonSim.num_50))
PearsonSim = PearsonSim.withColumn('Sim',PearsonSim.Pearson_k * PearsonSim.PearsonSim)\
                                   .select('user_id', 'PearsonSim', 'Sim')
PearsonSim.show(10)

+-------+--------------------+--------------------+
|user_id|          PearsonSim|                 Sim|
+-------+--------------------+--------------------+
|     26|  0.3802046010514601|  0.3802046010514601|
|     29|   0.668526749890325|   0.347633909942969|
|    474| 0.47907547058703714| 0.47907547058703714|
|     65|  0.0364400268215457|0.035711226285114785|
|    191|  0.4263378766068558| 0.17906190817487944|
|    418|  0.2718453302625679| 0.08155359907877037|
|    541| 0.38378416956628664|   0.345405752609658|
|    558|-0.21268253840976273|-0.07231206305931934|
|    222|  0.5468473187813252|  0.5468473187813252|
|    270| 0.12321436546386905| 0.12321436546386905|
+-------+--------------------+--------------------+
only showing top 10 rows



In [256]:
sortedPearson = PearsonSim.orderBy(PearsonSim['Sim'].desc())
sortedPearson[sortedPearson.Sim<1].show(10)

+-------+------------------+------------------+
|user_id|        PearsonSim|               Sim|
+-------+------------------+------------------+
|    397|0.7764595887588565|0.7764595887588565|
|     21| 0.736772656463565| 0.736772656463565|
|    303|0.7208205983881428|0.7208205983881428|
|    615|0.6898508489153302|0.6898508489153302|
|    276|0.6689012425451382|0.6689012425451382|
|    453|0.6643416809296162|0.6643416809296162|
|    595|0.6448655928460737|0.6448655928460737|
|    267|0.6365630482981355|0.6365630482981355|
|    268| 0.635586446428676| 0.635586446428676|
|    890|0.5843180584673966|0.5843180584673966|
+-------+------------------+------------------+
only showing top 10 rows



In [257]:
pearson_neighbours = sortedPearson[sortedPearson.Sim<1].limit(30) 

In [331]:
items = df_data.select('item_id').distinct()
items = items.join(df_my_user,items.item_id==df_my_user.my_item_id,'left_outer')\
        .filter(F.isnull('my_user_rating'))\
        .selectExpr('item_id as items_id')

In [332]:
df_pred = df_data.join(items, df_data.item_id==items.items_id)\
            .filter(df_data['user_id'].isin(pearson_neighbours['user_id']))\
            .select('user_id','item_id','rating')
df_pred.show(5)

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
|    138|     26|     5|
|    224|     26|     3|
|     18|     26|     4|
|    222|     26|     3|
|     43|     26|     5|
+-------+-------+------+
only showing top 5 rows



In [342]:
df_pred = df_pred.join(avg_user_rating, df_pred.user_id==avg_user_rating.user)\
            .select('*', (F.col('rating')-F.col('avg_user_rating')).alias('diff_user_r'))\
            .join(pearson_neighbours, df_pred.user_id==pearson_neighbours.user_id)\
            .withColumn('mult_diff_sim',F.col('diff_user_r') * F.col('Sim'))\
            .withColumn('abs_sim', F.abs(F.col('Sim')))\
            .select('user','item_id','mult_diff_sim','abs_sim')

In [349]:
df_pred = df_pred.groupby('item_id').agg(F.sum(F.col('mult_diff_sim')).alias('sum_1'),
                            F.sum(F.col('abs_sim')).alias('sum_2'))\
            .withColumn('pred', 3.813888888888889 - F.col('sum_1') / F.col('sum_2'))\
            .orderBy(F.col('pred').desc()).limit(10)\
            .select('item_id','pred')

In [358]:
[int(row.user_id) for row in pearson_neighbours.select('user_id').collect()]

[397,
 21,
 303,
 615,
 276,
 453,
 595,
 267,
 268,
 890,
 526,
 345,
 851,
 916,
 292,
 933,
 69,
 823,
 293,
 22,
 94,
 394,
 1,
 715,
 246,
 457,
 222,
 249,
 379,
 291]

In [356]:
[int(row.item_id) for row in df_pred.select('item_id').collect()]

[1128, 894, 555, 398, 1001, 112, 104, 1215, 1274, 352]

# Часть 3. Базовые предикторы

Глобальное среднее (average_rating) по всему датасету. Сумма всех оценок по всем фильмам/Количество всех оценок по всем фильмам. 

In [385]:
average_rating = df_data.agg(F.count('rating').alias('rating_count'),\
                F.sum('rating').alias('rating_sum'))\
                .withColumn('glob_rating_avg', F.col('rating_sum') / F.col('rating_count'))\
                .head()[2]
average_rating

3.52986

Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь Ia - множество фильмов, по которым у пользователя есть рейтинги, а |Ia| их количество.

In [430]:
df_bu = df_data.withColumn('r_u-m', F.col('rating')-average_rating)\
               .groupby('user_id').agg(F.sum(F.col('r_u-m')).alias('sum_r_u-m'),\
                               F.count(F.col('item_id')).alias('count_item'))\
               .withColumn('Bu', F.col('sum_r_u-m')/(F.col('count_item')+10))
df_bu.show(5)

+-------+------------------+----------+-------------------+
|user_id|         sum_r_u-m|count_item|                 Bu|
+-------+------------------+----------+-------------------+
|     26|-62.69501999999996|       107|-0.5358548717948715|
|     29|3.9847599999999925|        34| 0.0905627272727271|
|    474|180.73578000000018|       327| 0.5363079525222557|
|    541|12.528620000000004|       133| 0.0876127272727273|
|    558|13.402799999999996|        20| 0.4467599999999999|
+-------+------------------+----------+-------------------+
only showing top 5 rows



Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь Ui - множество пользователей, которые оценили данный фильм, а |Ui| их количество.

In [426]:
df_bi = df_base.join(df_bu, ['user_id'])\
            .withColumn('r_i-m-bu', F.col('rating')-F.col('Bu')-average_rating)\
            .groupby('item_id').agg(\
                            F.sum(F.col('r_i-m-bu')).alias('sum_r_u-m-Bu'),\
                            F.count(F.col('item_id')).alias('count_user'))\
            .withColumn('Bi', F.col('sum_r_u-m-Bu')/(F.col('count_user')+25))
                                    
df_bi.show(5)

+-------+--------------------+----------+--------------------+
|item_id|        sum_r_u-m-Bu|count_user|                  Bi|
+-------+--------------------+----------+--------------------+
|    474|   122.5824431825819|       194|  0.5597371834821091|
|     26|  -8.263462025852998|        73|-0.08432104108013264|
|     29|  -89.49177232307981|       114| -0.6438257001660418|
|   1677|-0.22598502202643234|         1|-0.00869173161640...|
|    964|-0.22889681260457673|         9|-0.00673225919425...|
+-------+--------------------+----------+--------------------+
only showing top 5 rows



In [435]:
df_base = df_data.join(df_bu.select('user_id','Bu'),['user_id'])\
       .join(df_bi.select('item_id','Bi'),['item_id'])\
    .withColumn('Bui', average_rating+F.col('Bu')+(F.col('Bi')))
df_base.show(5)   

+-------+-------+------+--------------------+--------------------+------------------+
|item_id|user_id|rating|                  Bu|                  Bi|               Bui|
+-------+-------+------+--------------------+--------------------+------------------+
|     26|    299|     4|-0.06331310344827543|-0.08432104108013265|3.3822258554715923|
|     26|    593|     4| 0.07434843373493996|-0.08432104108013265|3.5198873926548075|
|     26|    478|     5|-0.10466813008130096|-0.08432104108013265|3.3408708288385665|
|     26|    194|     3| -0.5479596825396832|-0.08432104108013265|2.8975792763801844|
|     26|    151|     3| 0.45215451104101007|-0.08432104108013265|3.8976934699608776|
+-------+-------+------+--------------------+--------------------+------------------+
only showing top 5 rows



# Часть 4. Item-item CF

In [436]:
df_base = df_base.withColumn('r-Bui', F.col('rating')-F.col('Bui'))
df_base.show(5)

+-------+-------+------+-------------------+--------------------+------------------+--------------------+
|item_id|user_id|rating|                 Bu|                  Bi|               Bui|               r-Bui|
+-------+-------+------+-------------------+--------------------+------------------+--------------------+
|     26|    399|     2|-0.5745451063829782|-0.08432104108013262|2.8709938525368894| -0.8709938525368894|
|     26|    405|     3|-1.6726998929049472|-0.08432104108013262|1.7728390660149203|  1.2271609339850797|
|     26|      1|     3|0.07758184397163108|-0.08432104108013262|3.5231208028914986| -0.5231208028914986|
|     26|    682|     3|-0.3824306601466988|-0.08432104108013262|3.0631082987731686|-0.06310829877316859|
|     26|    758|     4| 0.3374386376021804|-0.08432104108013262| 3.782977596522048|   0.217022403477952|
+-------+-------+------+-------------------+--------------------+------------------+--------------------+
only showing top 5 rows

